In [1]:
import pandas as pd
import unidecode
import string
import sklearn.feature_extraction
import itertools
from langdetect import detect
from langdetect import DetectorFactory
#DetectorFactory.seed = 0
from tqdm import tqdm
import re
import seaborn as sns
from collections import defaultdict
from collections import Counter
import matplotlib.pyplot as plt
import nltk
from normalise import normalise
import numpy as np

/Users/tomashegewisch/research_project/Tomas/env/lib/python3.8/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.semi_supervised.label_propagation module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.semi_supervised. Anything that cannot be imported from sklearn.semi_supervised is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/tomashegewisch/research_project/Tomas/env/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LabelPropagation from version 0.18 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [2]:
rm_punc = str.maketrans(string.punctuation, ' ' * len(string.punctuation))
stop_words = sklearn.feature_extraction.text.ENGLISH_STOP_WORDS

In [3]:
temp_word = set(stop_words)
temp_word.add("RT") #<<< ADD MORE WORDS>>>
temp_word.add("RT")
temp_word.add("The")
temp_word.add("new")
temp_word.add("-")
temp_word.add("I")
temp_word.add(".")
temp_word.add("virus")
temp_word.add("http")
temp_word.add("...")
stop_words = frozenset(temp_word)

In [4]:
tweets = pd.read_json('/Users/tomashegewisch/Downloads/030110/LIM368_20200307.json', lines=True, orient='record')
#tweets = pd.read_json('/Users/tomashegewisch/Downloads/030110/test.json', lines=True, orient='record')

pd.set_option('display.max_rows', tweets.shape[0]+1)

In [5]:
print("BEFORE")
print(len(tweets))
tweets.drop_duplicates(subset=['id'], keep="first", inplace=True)
print("\nAFTER")
print(len(tweets))

BEFORE
1117

AFTER
1117


In [6]:
#tweets.columns


In [7]:
tweets['tweet'][3]

'Since we no longer doing the same slots. I get a chance to listen to u 🙂'

In [8]:
def clean_tweet_word(word):
    if (len(word) > 1) and (word[0] == '#' or word[0] == '@'):
        return word[0] + word[1:].translate(rm_punc).strip()
    if word.startswith('http') or word.startswith('pic.twitter.com') or word.endswith('.com') or word.endswith('.co.za'):
        return ''
    if word in stop_words:
        return ''
    return word.translate(rm_punc).strip()

def clean_tweet(tweet):
    tweet = unidecode.unidecode(tweet).lower().split()
    tweet = [clean_tweet_word(x) for x in tweet]
    tweet = list(itertools.chain.from_iterable([x.split() for x in tweet if x != '']))
    tweet = [x for x in tweet if len(x) > 1]
    return tweet

In [9]:
# we want to get rid of all the tweets that do not have english in them
print("The length of the DF(before): " + str(len(tweets)))
non_text = 0
non_en = 0
temp_list_non_eng = []
for i in tqdm(tweets['tweet'].tolist()):
    try:
        value = detect(i)
        if value != "en":
            temp_list_non_eng.append(i)
            #print(value+"----" + i)
    except:
        non_text +=1
print("Non-Recognisable text: " + str(non_text))        
print("we found: "+str(len(temp_list_non_eng))+ " non-eng tweets")

for i in temp_list_non_eng:
    tweets = tweets[tweets.tweet != i]
    
print("The length of the DF(after): " + str(len(tweets)))

  0%|          | 0/1117 [00:00<?, ?it/s]

The length of the DF(before): 1117


100%|██████████| 1117/1117 [00:29<00:00, 38.01it/s]


Non-Recognisable text: 166
we found: 339 non-eng tweets
The length of the DF(after): 778


In [10]:
# phone number
#txt = "The rain in Spain 0218879544"
#re.findall("\d+", txt)
for i in tweets['tweet']:
    for j in re.findall("\d+", i):
        if len(j) == 10:
            print(j)

1477000386
0794172654
0734307920


In [11]:
tokinized = tweets['tweet'].apply(clean_tweet)

In [12]:
tfidf_transform = sklearn.feature_extraction.text.TfidfVectorizer(analyzer=clean_tweet, min_df=0.01)
X = tfidf_transform.fit_transform(tweets['tweet'])

In [13]:
tfidf_transform.get_feature_names()

['#absaprem',
 '#eababanights',
 '#ultimateloveng',
 'africa',
 'baroka',
 'black',
 'campus',
 'can',
 'come',
 'day',
 'did',
 'don',
 'eish',
 'fc',
 'good',
 'hope',
 'it',
 'just',
 'kids',
 'know',
 'leopards',
 'let',
 'like',
 'limpopo',
 'love',
 'mall',
 'need',
 'north',
 'people',
 'polokwane',
 're',
 'say',
 'team',
 'thank',
 'that',
 'time',
 'today',
 'turfloop',
 'win',
 'ya',
 'yes',
 'you']

In [14]:
#tokinized
tweets['tokinized'] = tokinized

In [15]:
print(tweets['tweet'][1000])
print(tweets['tokinized'][1000])

#KeaDrive
['#keadrive']


In [16]:
# importing NLTK stuff manualy
# import nltk
# import ssl

# try:
#     _create_unverified_https_context = ssl._create_unverified_context
# except AttributeError:
#     pass
# else:
#     ssl._create_default_https_context = _create_unverified_https_context

# nltk.download()

In [17]:
# text = ["On", "the", "28", "Apr.", "2010", ",", "Dr.", "Banks", "bought", "a", "chair", "for", "£35", "."]
# normalised_words = normalise(text, verbose=False)
# normalised_words

In [18]:
def normalise_func(tok_words_list):
    #print(tok_words_list)
    return normalise(tok_words_list, verbose=False)

In [19]:
normalised = tweets['tokinized'].apply(normalise_func)

In [22]:
tweets['normalised'] = normalised